## Track 2 (aggregated data)

In [1]:
import os
import json
import copy
import pyhrv
import scipy
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from monai.config import KeysCollection
from monai.transforms import MapTransform
from monai.transforms import Compose, ToTensorD
from monai.data import CacheDataset, DataLoader, DistributedSampler

valid_range = {
    "acc_X" : (-19.6, 19.6),
    "acc_Y" : (-19.6, 19.6),
    "acc_Z" : (-19.6, 19.6),
    "gyr_X" : (-573, 573),
    "gyr_Y" : (-573, 573),
    "gyr_Z" : (-573, 573),
    "heartRate" : (0, 255),
    "rRInterval" : (0, 2000),
}

/home/s_calcagno/miniconda3/envs/pytorch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def get_valid_mask(slice, keys):
    valid_mask = pd.Series([True]*len(slice))
    valid_mask.index = slice.index
    for k in keys:
        a, b = valid_range[k]
        valid_mask = valid_mask & (slice[k]>a) & (slice[k]<=b)
    return valid_mask

def mean_acc_norm(slice):
    # Compute the mean norm of accelerometer
    keys = ['acc_X', 'acc_Y', 'acc_Z']
    valid_mask = get_valid_mask(slice, keys)
    if not valid_mask.any():
        return np.nan
    acc_norm = slice[valid_mask][keys].apply(lambda x: np.linalg.norm(x), axis=1).to_numpy()
    return acc_norm.mean()

def mean_gyr_norm(slice):    
    # Compute the mean norm of gyroscope
    keys = ['gyr_X', 'gyr_Y', 'gyr_Z']
    valid_mask = get_valid_mask(slice, keys)
    if not valid_mask.any():
        return np.nan
    gyr_norm = slice[valid_mask][keys].apply(lambda x: np.linalg.norm(x), axis=1).to_numpy()
    return gyr_norm.mean()

def mean_hrv(slice):
    # Compute the mean HRV
    key = 'heartRate'
    valid_mask = get_valid_mask(slice, [key])
    if not valid_mask.any():
        return np.nan
    return slice[valid_mask][key].mean()

def mean_rr(slice):
    # Compute the mean RR interval
    key = 'rRInterval'
    valid_mask = get_valid_mask(slice, [key])
    if not valid_mask.any():
        return np.nan
    return slice[valid_mask][key].mean()

def poincare_major_axis(slice):
    # Compute the Major Axis of Poincarè Plot
    keys = ['rRInterval']
    valid_mask = get_valid_mask(slice, keys)
    if not valid_mask.any():
        return np.nan
    plt.ioff()
    return pyhrv.nonlinear.poincare(slice[valid_mask][keys].to_numpy(), show=False)['sd2']

def lombscargle_power(slice):
    # Compute the Low  and High Frequency Power of Lomb-Scargle Periodogram
    key = 'rRInterval'
    valid_mask = get_valid_mask(slice, [key])
    if not valid_mask.any():
        return np.nan, np.nan
    nni = slice[valid_mask][key]
    # low frequencies
    l = 0.04 * np.pi /2
    h = 0.15 * np.pi /2
    freqs = np.linspace(l, h, 1000)
    lf_lsp = scipy.signal.lombscargle(nni.to_numpy(), nni.index.to_numpy(), freqs, normalize=True)
    # high frequencies
    l = 0.15 * np.pi /2
    h = 0.4 * np.pi /2
    freqs = np.linspace(l, h, 1000)
    hf_lsp = scipy.signal.lombscargle(nni.to_numpy(), nni.index.to_numpy(), freqs, normalize=True)
    return np.trapz(lf_lsp, freqs), np.trapz(hf_lsp, freqs)

def time_encoding(slice):
    # Compute the sin and cos of timestamp (we have 12*24=288 5-minutes per day)
    mean_timestamp = slice['timecol'].astype('datetime64').mean()
    h = mean_timestamp.hour
    m = mean_timestamp.minute
    time_value = h*60 + m
    sin_t = np.sin(time_value*(2.*np.pi/(60*24)))
    cos_t = np.cos(time_value*(2.*np.pi/(60*24)))
    return sin_t, cos_t

def validity_percentage(slice):
    # Compute the % of valid data
    validity = []
    for k, (a, b) in valid_range.items():
        valid_mask = (slice[k] > a) & (slice[k] <= b)
        try:
            num_valid = valid_mask.value_counts()[True]
        except KeyError:
            num_valid = 0
        percent_valid = num_valid/len(valid_mask)
        validity.append(percent_valid)
        return np.array(validity).mean()

def extract_features(slice, return_numpy=False):
    if return_numpy:
        return np.array([
            mean_acc_norm(slice),
            mean_gyr_norm(slice),
            mean_hrv(slice),
            mean_rr(slice),
            poincare_major_axis(slice),
            *lombscargle_power(slice),
            *time_encoding(slice),
            validity_percentage(slice),
        ])
    l, h = lombscargle_power(slice)
    sin, cos = time_encoding(slice)
    return {
            'mean_acc_norm' : mean_acc_norm(slice),
            'mean_gyr_norm' : mean_gyr_norm(slice),
            'mean_hrv' : mean_hrv(slice),
            'mean_rr' : mean_rr(slice),
            'poincare_major_axis' : poincare_major_axis(slice),
            'lombscargle_power_low' : l,
            'lombscargle_power_high' : h,
            'sin_time_encoding' : sin,
            'cos_time_encoding' : cos,
            'validity_percentage' : validity_percentage(slice),
        }

In [3]:
root_dir = Path("../../datasets/SPGC_challenge_track_2_release/")

def get_paths(root_dir, split):
    paths = []
    if split != 'test':
        base_dir = Path('training_data')
        for user in os.listdir(root_dir/base_dir):
            user_dir = base_dir/Path(user)/Path(split)
            for status in os.listdir(root_dir/user_dir):
                status_dir = user_dir/Path(status)
                for sample in os.listdir(root_dir/status_dir):
                    paths.append(status_dir/Path(sample))
    else:
        base_dir = Path('test_data')
        for user in os.listdir(root_dir/base_dir):
            user_dir = base_dir/Path(user)/Path(split)
            for sample in os.listdir(root_dir/user_dir):
                paths.append(user_dir/Path(sample))
    return paths

def parse_path(path):
    path = str(path).split("/")
    user = int(path[1].split("_")[1])
    split = path[2]
    if len(path)==5:
        status = 1 if path[3]=='relapse' else 0
        id = int(path[4])
    else:
        status = -1
        id = int(path[3])
    return user, split, status, id

In [4]:
paths = {split: get_paths(root_dir, split) for split in ['train', 'val', 'test']}

for split in ['train', 'val', 'test']:
    for sample in tqdm(paths[split]):
        if len(os.listdir(root_dir/sample)) < 3:
            data = pd.read_csv(root_dir/sample/"data.csv")
            # interate over data
            slices = []
            for start in range(0, len(data), 12*5):
                # Extract a 5-minutes slices
                slice = data.loc[start:start+59]
                slices.append(extract_features(slice))
            pd.DataFrame().from_records(slices).to_csv(root_dir/sample/"aggregated_data.csv")

  0%|          | 0/1906 [00:00<?, ?it/s]

  0%|          | 0/531 [00:00<?, ?it/s]

  0%|          | 0/544 [00:00<?, ?it/s]

In [5]:
base_dir = Path('training_data')
split = 'val'
for user in os.listdir(root_dir/base_dir):
    user_dir = base_dir/Path(user)/Path(split)
    for status in os.listdir(root_dir/user_dir):
        print(f"User: {user}, Status: {status}, Samples:{len(os.listdir(root_dir/user_dir/Path(status)))}")

User: user_02, Status: non-relapse, Samples:25
User: user_02, Status: relapse, Samples:13
User: user_08, Status: non-relapse, Samples:13
User: user_08, Status: relapse, Samples:3
User: user_00, Status: non-relapse, Samples:31
User: user_00, Status: relapse, Samples:9
User: user_01, Status: non-relapse, Samples:22
User: user_01, Status: relapse, Samples:57
User: user_09, Status: non-relapse, Samples:21
User: user_09, Status: relapse, Samples:73
User: user_05, Status: non-relapse, Samples:27
User: user_05, Status: relapse, Samples:22
User: user_07, Status: non-relapse, Samples:29
User: user_07, Status: relapse, Samples:93
User: user_06, Status: non-relapse, Samples:26
User: user_06, Status: relapse, Samples:4
User: user_04, Status: non-relapse, Samples:22
User: user_04, Status: relapse, Samples:3
User: user_03, Status: non-relapse, Samples:21
User: user_03, Status: relapse, Samples:17


### Create Dataset Dataframes

In [6]:
root_dir = Path("../../datasets/SPGC_challenge_track_2_release/")
splits = ['train', 'val', 'test']
paths = {split: get_paths(root_dir, split) for split in splits}

def validate(window):
    invalid_filter = window.isna().any(axis=1)
    return 1- (len(window[invalid_filter])/len(window)) 

def get_observations(root_dir, path, w_size_h=4, w_stride_h=1, val_percentage=0.25):
    
    data = pd.read_csv(root_dir/path/"aggregated_data.csv")
    user, split, status, id = parse_path(path)
    w_size = int(w_size_h*12)
    w_stride = int(w_stride_h*12)
    obs = []
    path = Path(path/"aggregated_data.csv")
    # Treat short sequences
    if len(data) < w_size:
        if split == 'train':
            return obs
        # Consider short windows in validation and test
        else:
            validity = validate(data)
            return [{
                'data_file' : path,
                'user_id' : user,
                'sample_id' : id,
                'label' : status,
                'valid' : validity >= val_percentage,
                'start_data_row' : 0,
                'end_data_row' : len(data) 
            }]
    
    # Slide windows
    for start in range(0, len(data)-w_size, w_stride):
        stop = start + w_size # excluded
        window = data.loc[start:stop-1] # upperbound is included
        # check validity
        validity = validate(window)
        obs.append({
            'data_file' : path,
            'user_id' : user,
            'sample_id' : id,
            'label' : status,
            'valid' : validity >= val_percentage,
            'start_data_row' : start,
            'end_data_row' :stop
        })

    return obs

def create_dataset_list(root_dir, paths,  w_size_h=4, w_stride_h=1, val_percentage=0.25):
    dataset_list = []
    for sample in paths:
        # open file
        dataset_list.extend(get_observations(root_dir, sample, w_size_h=w_size_h, w_stride_h=w_stride_h, val_percentage=val_percentage))
    return dataset_list

def _create_offsets(x):
    if len(x[x.valid]) == 0:
        return list(zip(x.start_data_row, x.end_data_row))
    return list(zip(x[x.valid].start_data_row, x[x.valid].end_data_row))

def save_dataset(root_dir, output_dir, w_size_h=4, w_stride_h=1, val_percentage={'train': 2.5/3, 'val':1/3, 'test':1/3}):
    for split in splits:
        # create records
        dataset_list = create_dataset_list(root_dir, paths[split], w_size_h=w_size_h, w_stride_h=w_stride_h, val_percentage=val_percentage[split])
        # create dataframe
        dataset = pd.DataFrame(dataset_list)
        if split != 'train':
            # group by sample_id (data_file) and create a list of valid offsets
            records = dataset.groupby('data_file').apply(lambda x: {
                    'data_file' : x.data_file.iloc[0],
                    'user_id' : x.user_id.iloc[0],
                    'sample_id' : x.sample_id.iloc[0],
                    'label' : x.label.iloc[0],
                    'valid' : 1,
                    'offsets' : _create_offsets(x),
                })
            dataset = pd.DataFrame().from_records(records.to_list())
        dataset.to_csv(output_dir/f"{split}_dataset.csv")

In [8]:
root_dir = Path("../../datasets/SPGC_challenge_track_2_release")
output_dir = Path("../data/track2/aggregated_volund")
os.makedirs(output_dir, exist_ok=True)

w_size_h = 5.334
w_stride_h = 1
val_percentage = {'train': 2.5/3, 'val':1/3, 'test':1/3}

save_dataset(root_dir, output_dir, w_size_h=w_size_h, w_stride_h=w_stride_h, val_percentage=val_percentage)

### Compute per-subject statistics

In [9]:
# Compute Stats
root_dir = Path("../../datasets/SPGC_challenge_track_2_release/training_data")
stats = {}

for user in os.listdir(root_dir):
    user_id = int(user.split("_")[1])
    user_dir = root_dir/Path(f"{user}/train/non-relapse")
    user_dfs = []
    for sample in os.listdir(user_dir):
        # read aggregated data
        user_dfs.append(pd.read_csv(user_dir/Path(sample)/"aggregated_data.csv", index_col=0))
    # concatenate dataframes
    df = pd.concat(user_dfs).replace([np.inf, -np.inf], np.nan)
    mean = df.mean()
    std = df.std()
    record = pd.concat([mean, std], axis=1)
    record.columns = ['mean', 'std']
    stats[user_id] = record.transpose().to_dict()

{'mean': 0.4846786305433228, 'std': 0.3840024955591947}


In [4]:
output_dir = Path("../data/track2/")

#with open(output_dir/"subject_stats.json", "w") as f:
#    json.dump(stats, f)

with open(output_dir/"subject_stats.json", "r") as f:
    stats = json.load(f)

user = '9'
print(stats[user]['mean_acc_norm'])

{'mean': 0.4846786305433228, 'std': 0.3840024955591947}


### Dataset class and Transforms

In [6]:
class EPreventionDataset(CacheDataset):
    def __init__(self, split_path, split, transforms, max_samples=None, subject=None, cache_num = sys.maxsize, cache_rate=1.0, num_workers=1):    
        
        self.split = split
        self.max_samples = max_samples
        
        data = self._generate_data_list(split_path/f"{split}_dataset.csv", subject=subject)

        super().__init__(data, transforms, cache_num=cache_num, cache_rate=cache_rate, num_workers=num_workers)
        
     
    #split data in train, val and test sets in a reproducible way
    def _generate_data_list(self, split_path, subject = None):

        # open csv with observations
        data_list = pd.read_csv(split_path, index_col=0, nrows=self.max_samples)
        if subject is not None:
           # filter subject
            data_list = data_list[data_list['user_id']==subject]
        # filter valid
        data_list = data_list[data_list.valid.astype(bool)]
        # save ditribution
        count_distribution = data_list.label.value_counts().sort_index().to_numpy()
        num_samples = len(data_list)
        self.distribution = count_distribution / num_samples

        return data_list.to_dict('records')  
    
    def get_label_proportions(self):

        return self.distribution


class AppendRootDirD(MapTransform):

    def __init__(self, keys: KeysCollection, root_dir):
        super().__init__(keys)
        self.root_dir = root_dir
    
    def __call__(self, data):
        d = copy.deepcopy(data)
        for k in self.keys:
            d[k] = os.path.join(self.root_dir,d[k])
        return d

class LoadAggregatedDataD(MapTransform):
    
    def __init__(self, keys: KeysCollection, split):
        super().__init__(keys)
        self.split = split

    def __call__(self, data):
        d = copy.deepcopy(data)
        for k in self.keys:
            if self.split == 'train':
                d['data'] = pd.read_csv(d[k],
                    skiprows=lambda x : x in range(1, d['start_data_row']+1),
                    nrows=d['end_data_row']-d['start_data_row']
                ) 
            else:
                d['data'] = pd.read_csv(d[k])
            del d[k]
            del d['data']['Unnamed: 0']
        if 'valid' in d.keys(): del d['valid']
        if 'start_data_row' in d.keys(): del d['start_data_row']
        if 'end_data_row' in d.keys(): del d['end_data_row']
        return d

class ImputeMedianD(MapTransform):
    
    def __init__(self, keys: KeysCollection):
        super().__init__(keys)

    def __call__(self, data):
        d = copy.deepcopy(data)
        for k in self.keys:
            # impute median
            d[k] = d[k].replace([np.inf, -np.inf], np.nan)
            d[k] = d[k].fillna(d[k].median())
            # check whole nan cols
            user = str(d['user_id'])
            for col in d[k].columns:
                if d[k][col].isna().all():
                    d[k][col] = stats[user][col]['mean']
        return d

class ToNumpyD(MapTransform):
    
    def __init__(self, keys: KeysCollection):
        super().__init__(keys)

    def __call__(self, data):
        d = copy.deepcopy(data)
        for k in self.keys:
            d[k] = d[k].to_numpy()
        return d

class StandardizeD(MapTransform):
    
    def __init__(self, keys: KeysCollection):
        super().__init__(keys)

    def __call__(self, data):
        d = copy.deepcopy(data)
        for k in self.keys:
            user = str(d['user_id'])
            means = torch.tensor([stat['mean'] for _, stat in stats[user].items()])
            stds = torch.tensor([stat['std'] for _, stat in stats[user].items()])
            means[7:] = 0.
            stds[7:] = 1.
            #print(means, stds)
            d[k] = (d[k] - means)/stds
        return d

class TransposeD(MapTransform):
    
    def __init__(self, keys: KeysCollection):
        super().__init__(keys)

    def __call__(self, data):
        d = copy.deepcopy(data)
        for k in self.keys:
            d[k] = d[k].t()
        return d

class FlattenD(MapTransform):
    
    def __init__(self, keys: KeysCollection):
        super().__init__(keys)

    def __call__(self, data):
        d = copy.deepcopy(data)
        for k in self.keys:
            if len(d[k].shape) == 2:
                d[k] = d[k].flatten()
            else:
                d[k] = d[k].flatten(start_dim=1)
        return d

root_dir = Path("../../datasets/SPGC_challenge_track_2_release")

transforms = [
        ToTensorD(['label'],dtype=torch.long),
        AppendRootDirD(['data_file'], root_dir),
        LoadAggregatedDataD(['data_file'], 'train'),
        ImputeMedianD(['data']),
        ToNumpyD(['data']),
        ToTensorD(['data'], dtype=torch.float),
        StandardizeD(['data']),
        TransposeD(['data']),
        FlattenD(['data'])
]

transforms = Compose(transforms)

train_data = EPreventionDataset(Path("../data/track2/"), 'train', transforms=transforms, subject=0)

Loading dataset: 100%|██████████| 2352/2352 [00:12<00:00, 194.24it/s]


### Add Transforms for Validation and Test

In [12]:
from torch.nn import ConstantPad1d, ReplicationPad1d

class PadShortSequenceD(MapTransform):
    
    def __init__(self, keys: KeysCollection, output_size, padding, mode):
        super().__init__(keys)
        assert padding in ['replication', 'zero'], "Select Proper Padding Mode: Allowed same and zero"
        assert mode in ['head', 'center', 'tail'], "Select Proper Mode: Allowed head, center and tail"
        self.output_size = output_size
        self.padding = padding
        self.mode = mode
        
    def __call__(self, data):
        d = copy.deepcopy(data)
        w_in = d['data'].shape[-1]
        if w_in >= self.output_size:
            return d
        pad_size = self.output_size - w_in
        if self.mode == 'head':
            padding = (pad_size, 0)
        elif self.mode == 'tail':
            padding = (0, pad_size)
        elif self.mode == 'center' and pad_size%2==0:
            padding = pad_size//2
        elif self.mode == 'center' and pad_size%2==1:
            padding = (pad_size//2, pad_size//2+1)
        pad_fn = self._get_pad_fn(padding)
        for k in self.keys:
            d[k] = pad_fn(d[k])
        return d

    def _get_pad_fn(self, padding):
        return ConstantPad1d(padding, 0) if self.padding == 'zero' else ReplicationPad1d(padding)

class CreateVotingBatchD(MapTransform):
    
    def __init__(self, keys: KeysCollection):
        super().__init__(keys)
        
    def __call__(self, data):
        d = copy.deepcopy(data)
        offsets = eval(d['offsets'])
        for k in self.keys:
            windows = [d[k][:, start:stop].unsqueeze(0) for (start, stop) in offsets]
            d[k] = torch.cat(windows, dim=0)
        if 'offsets' in d.keys():
            del d['offsets']
        return d

eval_transforms = [
        ToTensorD(['label'],dtype=torch.long),
        AppendRootDirD(['data_file'], root_dir),
        LoadAggregatedDataD(['data_file'], 'val'),
        ImputeMedianD(['data']),
        ToNumpyD(['data']),
        ToTensorD(['data'], dtype=torch.float),
        StandardizeD(['data']),
        TransposeD(['data']),
        CreateVotingBatchD(['data']),
        PadShortSequenceD(['data'], output_size=48, padding='replication', mode='head'),
        FlattenD(['data'])
]

eval_transforms = Compose(eval_transforms)

val_data = EPreventionDataset(Path("../data/track2/"), 'test', transforms=eval_transforms)

Loading dataset: 100%|██████████| 544/544 [00:05<00:00, 103.23it/s]


In [13]:
val_data[0]

{'user_id': 0,
 'sample_id': 0,
 'label': tensor(-1),
 'data': tensor([[-0.1862, -0.3519, -0.3272,  ...,  0.0069,  0.0069,  0.0069],
         [-0.3869, -0.3852, -0.3852,  ...,  0.0069,  0.0069,  0.0069],
         [-0.3750, -0.3554, -0.3515,  ...,  0.0069,  0.0069,  0.0069],
         ...,
         [-0.1448,  1.2524,  0.8345,  ...,  0.0069,  0.0069,  0.0069],
         [-0.1655,  1.0724,  2.1658,  ...,  0.0069,  0.0069,  0.0069],
         [ 0.5302,  1.9109,  0.8617,  ...,  0.0069,  0.0069,  0.0069]])}